## 9_4.転移学習による猫と犬の分類


In [8]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# データセットのダウンロード
# KaggleのDogs vs Catsデータをフィルタリングしたバージョンを使用
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

# データセットのデータアーカイブを"/.keras/datasets"ディレクトリにダウンロードし，
# アーカイブファイルのパスをpath?to?zipに格納，この時点でアーカイブは解凍される
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
# ファイルパスのファイル名を解凍後のフォルダー名'vats_and_dogs_filtered'に置き換え
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')



In [2]:
print(PATH)

/root/.keras/datasets/cats_and_dogs_filtered


In [3]:
%ls -la /root/.keras/datasets/cats_and_dogs_filtered

total 20
drwxr-xr-x 4 root root 4096 Aug 16 08:59 ./
drwxr-xr-x 3 root root 4096 Aug 16 08:59 ../
drwxr-xr-x 4 root root 4096 Aug 16 08:59 train/
drwxr-xr-x 4 root root 4096 Aug 16 08:59 validation/
-rw-r--r-- 1 root root  140 Aug 16 08:59 vectorize.py


In [9]:
# 訓練データ，検証データが格納されているディレクトリのファイルパスを変数に格納
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

# 訓練および検証データにおけるネコと犬のディレクトリを変数に格納
# 訓練用の猫画像のディレクトリ
train_cats_dir = os.path.join(train_dir, 'cats')
# 訓練用の犬画像のディレクトリ
train_dogs_dir = os.path.join(train_dir, 'dogs')

# 検証用の猫画像のディレクトリ
validation_cats_dir = os.path.join(validation_dir, 'cats')
# 検証用の犬画像のディレクトリ
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


In [5]:
print(train_cats_dir)

/root/.keras/datasets/cats_and_dogs_filtered/train/cats


In [10]:
# 画像の枚数を出力

# 訓練用の'cats'フォルダ
num_cats_tr = len(os.listdir(train_cats_dir))
print('training cat images:', num_cats_tr)

# 訓練用の'dogs'フォルダ
num_dogs_tr = len(os.listdir(train_dogs_dir))
print('training dog images:', num_dogs_tr)

# 検証用の'cats'フォルダ
num_cats_val = len(os.listdir(validation_cats_dir))
print('validation cat images:', num_cats_val)

# 検証用の'dogs'フォルダ
num_dogs_val = len(os.listdir(validation_dogs_dir))
print('validation dog images:', num_dogs_val)

# 訓練用の全ての画像
total_train = num_cats_tr + num_dogs_tr
print('Total training images:', total_train)

# 検証用の全ての画像
total_val = num_cats_val + num_dogs_val
print('Total validation images:', total_val)


training cat images: 1000
training dog images: 1000
validation cat images: 500
validation dog images: 500
Total training images: 2000
Total validation images: 1000


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ミニバッチのサイズ，リサイズ後の縦・横のサイズを登録
batch_size = 32
IMG_HEIGHT = 150
IMG_WIDTH = 150

# ImageDataGeneratorの生成
# 画像のピクセル値を０〜２５５の範囲から0.0〜１．０の範囲への正規化を指定
# 学習データ
train_image_generator = ImageDataGenerator(rescale=1./255)
# 検証データ
validation_image_generator = ImageDataGenerator(rescale=1./255)

# ImageDataGeneratorにミニバッチのサイズ，データのディレクトリ，
# リサイズ情報，正解ラベルの形式を設定してバッチデータ生成済みにする
#
# 訓練データ
train_data_gen = train_image_generator.flow_from_directory(
    batch_size=batch_size,         # ミニバッチのサイズ
    directory=train_dir,              # 抽出先のディレクトリ
    shuffle=True,                  # 抽出する際にシャッフルする
    target_size=(IMG_HEIGHT, IMG_WIDTH),  # 画像をリサイズ
    class_mode='binary'            # 正解ラベルを0と1にする
)

# 検証データ
val_data_gen = validation_image_generator.flow_from_directory(
    batch_size=batch_size,         # ミニバッチのサイズ
    directory=validation_dir,              # 抽出先のディレクトリ
    target_size=(IMG_HEIGHT, IMG_WIDTH),  # 画像をリサイズ
    class_mode='binary'            # 正解ラベルを0と1にする
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [14]:
# 正解ラベルを確認する
print(train_data_gen.class_indices)
print(val_data_gen.class_indices)

{'cats': 0, 'dogs': 1}
{'cats': 0, 'dogs': 1}


In [16]:
import matplotlib.pyplot as plt

# 訓練用のImageDataGeneratorを使ってミニバッチを抽出
train_batch, _ = next(train_data_gen)

# 5行4列のグリッドに画像をプロット
fig, axes = plt.subplots(5, 4, figsize=(12, 12))
axes = axes.flatten()
for img, ax in zip(train_batch, axes):
  ax.imshow(img)
  ax.axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.